In [1]:
from ipywidgets import interact, interactive, Image
import ipywidgets as widgets
from IPython.display import display, Markdown, IFrame

__author__ = "Miguel Terol Espino"
__date__ = "04/11/18"

<div style="text-align: center">
    <h1> BioInformatics 2018-2019 - Mini Project 2: Alignment and PSSM </h1>
</div>

## Part 1 : Data collection

### Part 1 - Method

The reduced file of BROMO Homo Sapiens sequences was created by doing a comparison of each sequence to each other and evaluating the avergae similarity. This method assures us that no sequence will be aligned that is similar to the whole set of other sequences, but leaves the possibility of some sequences still being rather similar.

A similar yet different method for the reduction would have been:
1. Randomly choose sequence from list of available and remove it from list of available
2. If set of chosen is empty, add to set and jump to 1, else continue
3. Compare chosen sequence to all sequences in set individually, if similarity always under 60%, add to set
4. Jump to 1

This method would assure that not a single sequence will have a more than 60% match, since exclusion is done on first case basis and not due to average, but it would introduce a random seed bias and the possibility of different sets for different runs.

Therefore to maintain stability and consensus of data accross all runs of the algorithms, some similarity possibility was sacrificed to maintain data that can be statically analyzed.

Similarity was compared by using the Levenshtein distance algorithm.

The class `FASTAParser` from the package `fastaParser` was created using the `biopython` package.

In [2]:
"""
FASTA file parser
"""

from Bio import SeqIO


class FASTAParser:
    """
    Parser for FASTA files
    """
    FORMAT = "fasta"

    def __init__(self):
        self.parse = SeqIO.parse

    def yield_parsed(self, filename):
        """Yield parsed sequences"""
        return [seq for seq in self.parse(filename, self.FORMAT)]


The similarity comparison was done using the `Levenshtein` package.

In [3]:
from fastaParser import FASTAParser
import Levenshtein


parser = FASTAParser()
sequences_gen = parser.yield_parsed("data/to-be-aligned.fasta")
sequences = []
sequences_id = []
for seq in sequences_gen:
    sequences.append(seq.seq)
    sequences_id.append(seq.id)

count = 0
with open("data/to-be-aligned-reduced.fasta", "w") as output_handle:
    for i in range(len(sequences)):
        sum = 0
        if i < len(sequences):
            for j in range(len(sequences)):
                if j != i:
                    ratio = Levenshtein.ratio(str(sequences[i]), str(sequences[j]))
                    sum += ratio
            if sum/(len(sequences) -i) <= 0.6:
                SeqIO.write((sequences_gen[i]),output_handle, "fasta")
            else:
                count += 1
    print(count, "sequences were removed from list")

144 sequences were removed from list


### Part 1 - Results

The files `to-be-aligned.fasta` and `to-be-aligned-reduced.fasta` can be found under the `data` directory.

## Part 2 : Multiple sequence alignment

### Part 2 - Method

The tool **CLUSTAL Omega** was used to align the sequences.

### Part 2 - Results

The files `msaresults-CLUSTALOmega.fasta` and `msaresults-reduced-CLUSTALOmega.fasta` can be found under the `data` directory.

## Part 3 : Profile construction

### Part 3 - Method

To create the PSSMs the `biopython` package was used.

The process was:

1. Read the aligned MSA using the `AlignIO` library
2. Get the alignment information using the `AlignInfo` library
3. Use the libraries method, `pos_specific_score_matrix` to construct our PSSM

Below each of the PSSMs the Weblogo constructed from the aligned MSA can be found.

In [4]:
from Bio import AlignIO
from Bio.Align import AlignInfo

def get_pssm(reducing):
    if reducing:
        filename = "data/msaresults-reduced-CLUSTALOmega.fasta"
        logo = "data/logo-msaresults-reduced-CLUSTALOmega.png" 
    else:
        filename = "data/msaresults-CLUSTALOmega.fasta"        
        logo = "data/logo-msaresults-CLUSTALOmega.png" 
    alignment = AlignIO.read(filename, "clustal")
    summary_align = AlignInfo.SummaryInfo(alignment)
    consensus = summary_align.dumb_consensus(threshold=0.5)
    my_pssm = summary_align.pos_specific_score_matrix(consensus, chars_to_ignore=["-"])
    print(my_pssm)
    display(Markdown("![Alt]({})".format(logo)))

interact(get_pssm, reducing=[('Reduced', True), ('Not reduced', False)]);


interactive(children=(Dropdown(description='reducing', options=(('Reduced', True), ('Not reduced', False)), va…

### Part 3 - Results

#### Part 3.1 : Weblogos

Weblogos where constructed through the the tool on the *threeplusone* website using the MSA results from **CLUSTAL Omga**

<div>
    <div id="notreduced" style="float: left; width: 45%;">
        <h4> Weblogo of unreduced sequences alignment</h4>
        <img src="data/logo-msaresults-CLUSTALOmega.png" >
        <p>
            From the results displayed on the weblogo we can see most conservation occurs around bits 145-195 with peaks at 45-65, 75-105 and 215-240. This corresponds to our PSSM matrix for the most part.
        </p>
    </div>
    <div id="reduced" style="float: right; width: 45%;">
        <h4> Weblogo of reduced sequences alignment</h4>
        <img src="data/logo-msaresults-reduced-CLUSTALOmega.png" >
        <p>
            From the results displayed on the weblogo we can see most conservation occurs around bits 140-185 with peaks at 85-100 and 205-235. This corresponds to our PSSM matrix for the most part. It would seem a few scores of 1.0 are being omitted in the begging of the logo, this could be due to a threshold parametrisation in the application or an error in our calculations.
        </p>
    </div>
</div>
<div style="clear: both;"></div>

#### Part 3.2 : HMM logo

HMM Bromo logo was obtained throught the resource provided at *pfam.xfam.org*. The logo is as follows:

![](data/HMM-Bromo.png)

The logo displays a clear similarity to our Weblogo of un-reduced sequences, whereas the reduced Weblogo has a few mismatchs,   introduced by creating our MSA on only a subset of the full family. On the other hand, the PSSM for the reduced sequences seems to follow more exactly the values of the HMM Logo.

## Part 4 : Aligning the profile to the sequences

### Part 4 - Method  and results

#### Part 4.1 : Linear penalty

Using the formula: $$ PEN'(p) = \frac{YMAX}{(OPEN + EXN*L)}$$

Where:
- $YMAX$ is the highest score in the PSSM
- $OPEN$ is the gap opening penalty
- $EXN$ is the gap extension penalty
- $L$ is number of residue positions in the gap

In [21]:
OPEN = 10
EXN = 0.5
def pen(pos, pssmMat):
    return pssmMat.max()/(OPEN + EXN*pssmMat.get_residue(p))    

#### Part 4.2 : Affine penalty

Using the formula: $$ PEN'(p) = \frac{YMAX}{(OPEN + EXN*LMAX(p))}$$

Where:
- $YMAX$ is the highest score in the PSSM
- $OPEN$ is the gap opening penalty
- $EXN$ is the gap extension penalty
- $LMAX$ is the length of the longest gap that includes p

In [23]:
OPEN = 5
EXN = 0.1
def pen(pos, pssmMat):
    return pssmMat.max()/(OPEN + EXN*pssmMat.get_longest_gap_length(p))

#### Part 4.3 : Protein sequence alignment

Motif scan with the Bromo PSSM gives:
- FT   MYHIT       831    848      
- FT   MYHIT      1221   1238

Meaning our domaines are found along those elements in traces.

#### Part 4.4 : UNIPROT verify

Alignment with **UNIPROT** gives:

    SP|P21675|TAF1_HUMAN MGPGCDLLLRTAATITAAAIMSDTDSDEDSAGGGPFSLAGFLFGNINGAGQLEGESVLDD 60
    SP|Q86U86|PB1_HUMAN  ------------------------------------------------------------

    SP|P21675|TAF1_HUMAN ECKKHLAGLGALGLGSLITELTANEELTGTDGALVNDEGWVRSTEDAVDYSDINEVAEDE 120
    SP|Q86U86|PB1_HUMAN  ------------------------------------------------------------

    SP|P21675|TAF1_HUMAN SRRYQQTMGSLQPLCHSDYDEDDYDADCEDIDCKLMPPPPPPPGPMKKDKDQDSITGEKV 180
    SP|Q86U86|PB1_HUMAN  ------------------------------------------------------------

    SP|P21675|TAF1_HUMAN DFSSSSDSESEMGPQEATQAESEDGKLTLPLAGIMQHDATKLLPSVTELFPEFRPGKVLR 240
    SP|Q86U86|PB1_HUMAN  ------------------------------------------------------------

    SP|P21675|TAF1_HUMAN FLRLFGPGKNVPSVWRSARRKRKKKHRELIQEEQIQEVECSVESEVSQKSLWNYDYAPPP 300
    SP|Q86U86|PB1_HUMAN  ------------------------------------------------------------

    SP|P21675|TAF1_HUMAN PPEQCLSDDEITMMAPVESKFSQSTGDIDKVTDTKPRVAEWRYGPARLWYDMLGVPEDGS 360
    SP|Q86U86|PB1_HUMAN  ------------------------------------------------------------

    SP|P21675|TAF1_HUMAN GFDYGFKLRKTEHEPVIKSRMIEEFRKLEENNGTDLLADENFLMVTQLHWEDDIIWDGED 420
    SP|Q86U86|PB1_HUMAN  ------------------------------------------------------------

    SP|P21675|TAF1_HUMAN VKHKGTKPQRASLAGWLPSSMTRNAMAYNVQQGFAATLDDDKPWYSIFPIDNEDLVYGRW 480
    SP|Q86U86|PB1_HUMAN  ------------------------------------------------------------

    SP|P21675|TAF1_HUMAN EDNIIWDAQAMPRLLEPPVLTLDPNDENLILEIPDEKEEATSNSPSKESKKESSLKKSRI 540
    SP|Q86U86|PB1_HUMAN  ------------------------------------------------------------

    SP|P21675|TAF1_HUMAN LLGKTGVIKEEPQQNMSQPEVKDPWNLSNDEYYYPKQQGLRGTFGGNIIQHSIPAVELRQ 600
    SP|Q86U86|PB1_HUMAN  ------------------------------------------------------------

    SP|P21675|TAF1_HUMAN PFFPTHMGPIKLRQFHRPPLKKYSFGALSQPGPHSVQPLLKHIKKKAKMREQERQASGGG 660
    SP|Q86U86|PB1_HUMAN  ------------------------------------------------------------

    SP|P21675|TAF1_HUMAN EMFFMRTPQDLTGKDGDLILAEYSEENGPLMMQVGMATKIKNYYKRKPGKDPGAPDCKYG 720
    SP|Q86U86|PB1_HUMAN  -----------------------------------MGSK------RRRATSP-------- 11
                                                            *.:*      *: ...*        
    SP|P21675|TAF1_HUMAN ETVYCHTSPFLGSLHPGQLLQAFENNLFRAPIYLHKMPETDFLIIRTRQGYYIRELVDIF 780
    SP|Q86U86|PB1_HUMAN  ------SSSVSGDFD--------------------------------------------- 20
                               :* . *.:.                                             
    SP|P21675|TAF1_HUMAN VVGQQCPLFEVPGPNSKRANTH-----------------IRDFL---QVFIYRLFWKSKD 820
    SP|Q86U86|PB1_HUMAN  -DG--HHSVSTPGPSRKRRRLSNLPTVDPIAVCHELYNTIRDYKDEQGRLLCELFIRAPK 77
                           *     ...***. ** .                   ***:      :: .** :: .
    SP|P21675|TAF1_HUMAN RPRRIRMEDIKKAFPS-HSESSIRKRLKLCADFKRTGMDSNWWVLKSDFRLPTEEEI--R 877
    SP|Q86U86|PB1_HUMAN  RRNQ---PDYYEVVSQPIDLMKIQQKLKME-EYD------DVNLLTADFQLLFNNAKSYY 127
                         * .:    *  :.. .  .  .*:::**:  ::.      :  :*.:**:*  ::     
    SP|P21675|TAF1_HUMAN AMVS--PEQCCAYYSMIAAEQRLKDAGYGEKSFFAPEEENEEDFQMKIDDEVRTAPWNTT 935
    SP|Q86U86|PB1_HUMAN  KPDSPEYKAACKLWDLYLRTR---------NEFVQKGEADDEDDDEDGQDNQGTVTEGSS 178
                            *   : .*  :.:    :         :.*.   * ::** : . :*:  *.  .::
    SP|P21675|TAF1_HUMAN RAFIAAMKGKCLLEVTGVADPTGCGEGFSYVKIPNKPTQQKDDKEPQPVKKTVTGTDADL 995
    SP|Q86U86|PB1_HUMAN  PAYLKEILEQLLEAIVVATNPSGRLISELFQKLPSKVQ---------------------- 216
                          *::  :  : *  :. .::*:*   .  : *:*.*                        
    SP|P21675|TAF1_HUMAN RRLSLKNAKQLLRKFGVPEEEIKKLSRWEVIDVVRTMSTEQARSGEGPMS---KFARGSR 1052
    SP|Q86U86|PB1_HUMAN  ----------------YP-------DYYAIIKEPIDLKTIAQRIQNGSYKSIHAMAKDID 253
                                          *       . : :*.    :.*   *  :*  .    :*:.  
    SP|P21675|TAF1_HUMAN FSVAEHQERYKEECQRIFDLQNKVLSSTEVLSTDTDSSSAEDSDFEEMGKNIENMLQNKK 1112
    SP|Q86U86|PB1_HUMAN  -LLAKNAKTYNEPGSQVFKDANSIKKIFYMKKA--------EIEHHEMAKSSLRM----R 300
                           :*:: : *:*  .::*.  *.: .   : .:        : :..**.*.  .*    :
    SP|P21675|TAF1_HUMAN TSSQLSREREEQERKELQRMLLAAGSAASGNNHRDDDTASVTSLNSSATGRCLKIYRTFR 1172
    SP|Q86U86|PB1_HUMAN  TPSNLA------------A------ARLTGPSHSKGS------LGE-ERNPTSKYYRNKR 335
                         * *:*:                   :  :* .* ...      *..   .   * **. *
    SP|P21675|TAF1_HUMAN DEEGKEYVRC---------------------ET-VRKPAVIDAYVRIRTTKDEEFIRKFA 1210
    SP|Q86U86|PB1_HUMAN  AVQGGRLSAITMALQYGSESEEDAALAAARYEEGESEAESITSFMDVS----NPF---YQ 388
                           :* .                         *    :   * ::: :     : *   : 
    SP|P21675|TAF1_HUMAN LFDEQHREEMRKERRRIQEQLRRLKRNQEKEKLKGPPEKKPKKMKERPDLKLKCGACGAI 1270
    SP|Q86U86|PB1_HUMAN  LYDT----------------VRSCRNNQ-GQLIA-----------------------EPF 408
                         *:*                 :*  :.**  : :                          :
    SP|P21675|TAF1_HUMAN GHMRTNKFCPLYYQTNAPPSNPVAMTEEQ-EEELEKTVIHND--------NEELIKVEGT 1321
    SP|Q86U86|PB1_HUMAN  YHLPSKKKYPDYYQQIKMPISLQQIRTKLKNQEYE-TLDHLECDLNLMFENAKRYNVPNS 467
                          *: ::*  * ***    * .   :  :  ::* * *: * :        * :  :* .:
    SP|P21675|TAF1_HUMAN KIVLGKQLIESADEVRRKSLVLKFPKQQLPPKKKRRVGTTVHC----D---YLNRPHKSI 1374
    SP|Q86U86|PB1_HUMAN  AIY--KRVLK-------LQQVMQAKKKELARRDDIEDGDSMISSATSDTGSAKRKSKKNI 518
                          *   *::::        . *::  *::*  :.. . * :: .    *     .: :*.*
    SP|P21675|TAF1_HUMAN HRRRTDPMVTLSSILESII-NDMRDLPNTYPFHTPVNAKVVKDYYKIITRPMDLQTLREN 1433
    SP|Q86U86|PB1_HUMAN  RKQRMKI--LFNVVLEAREPGSGRRLC--DLFMVKPSKKDYPDYYKIILEPMDLKIIEHN 574
                         :::* .    :. :**:   .. * *     * .  . *   ****** .****: :..*
    SP|P21675|TAF1_HUMAN VRKRLYPSREEFREHLELIVKNSATYNGPKHSLTQISQSMLDLCDEKLKEKEDKLARLEK 1493
    SP|Q86U86|PB1_HUMAN  IRNDKYAGEEGMIEDMKLMFRNARHYNEEGSQVYNDAHILEKL----LKEK-------RK 623
                         :*:  * ..* : *.::*:.:*:  **    .: : :: : .*    ****       .*
    SP|P21675|TAF1_HUMAN AINPLLDDDDQVAFSFILDNIV-------------TQKMM-------------AVPDSWP 1527
    SP|Q86U86|PB1_HUMAN  ELGPLPDDDDMASPKLKLSRKSGISPKKSKYMTPMQQKLNEVYEAVKNYTDKRGRRLSAI 683
                          :.** **** .: .: *..                **:              .   *  
    SP|P21675|TAF1_HUMAN FHHPVNKKFVPDYYKVIVNPMDLETIRKNISKHKYQSRESFLDDVNLILANSVKYNGPES 1587
    SP|Q86U86|PB1_HUMAN  FLRLPSRSELPDYYLTIKKPMDMEKIRSHMMANKYQDIDSMVEDFVMMFNNACTYNEPES 743
                         * :  .:. :**** .* :***:*.**.::  :***. :*:::*. ::: *: .** ***
    SP|P21675|TAF1_HUMAN QYTKTAQEIVNVCYQTLTEYDE----HLTQLE--------------------KDICTAKE 1623
    SP|Q86U86|PB1_HUMAN  LIYKDALVLHKVLLETRRDLEGDEDSHVPNVTLLIQELIHNLFVSVMSHQDDEGRCYSD- 802
                            * *  : :*  :*  : :     *: ::                     :. * :. 
    SP|P21675|TAF1_HUMAN AALEEAELESLDPMTPGPYTPQPPDLYDT---NTSLSMSRDASVFQDESNMSVLDIPSAT 1680
    SP|Q86U86|PB1_HUMAN  ---SLAEIPAVDPN----FPNKPPLTFDIIRKNVENNRYRRLDLFQEHM-FEVLERARR- 853
                            . **: ::**     :  :**  :*    *.. .  *  .:**:.  :.**:     
    SP|P21675|TAF1_HUMAN PEKQVTQEGEDGDGDLADEEEGTVQQPQASVLYEDLLMSEGEDDEEDAGSDEEGDNPFSA 1740
    SP|Q86U86|PB1_HUMAN  ----MN----RTDSEIYE---DAVELQQFFIKIRDELCKNGEILLS----------P--A 890
                             :.      *.:: :   .:*:  *  :  .* * .:**   .          *  *
    SP|P21675|TAF1_HUMAN IQLSESGSDSDVGSGGIRPKQPRMLQENTR-MDMENEESMMSYEGDGGEASHGLEDSNIS 1799
    SP|Q86U86|PB1_HUMAN  LSYTTKHLHNDVEKE-RKEKLPKEIEEDKLKREEEKREAEKSEDSSGAAGLSGLHR---T 946
                         :. : .  ..** .   : * *: ::*:.   : *:.*:  * :..*. .  **.    :
    SP|P21675|TAF1_HUMAN YGSYEEPDPKSNTQDTSFSSIGGYEVSEEEEDEEEEEQRSGPSVLSQVHLSEDEED---- 1855
    SP|Q86U86|PB1_HUMAN  YSQ-------DCSFKNSMYHVGDYVYVEPAEA------NLQPHIVCIERLWEDSAGEKWL 993
                         *..       . : ..*:  :*.*   *  *       .  * ::.  :* **. .    
    SP|P21675|TAF1_HUMAN --------SEDFHS-----------------------IAGDS------------------ 1866
    SP|Q86U86|PB1_HUMAN  YGCWFYRPNETFHLATRKFLEKEVFKSDYYNKVPVSKILGKCVVMFVKEYFKLCPENFRD 1053
                                 .* **                        * *..                  
    SP|P21675|TAF1_HUMAN --------------------------------------------------------DLDS 1870
    SP|Q86U86|PB1_HUMAN  EDVFVCESRYSAKTKSFKKIKLWTMPISSVRFVPRDVPLPVVRVASVFANADKGDDEKNT 1113
                                                                                 : ::
    SP|P21675|TAF1_HUMAN DE---------------------------------------------------------- 1872
    SP|Q86U86|PB1_HUMAN  DNSEDSRAEDNFNLEKEKEDVPVEMSNGEPGCHYFEQLHYNDMWLKVGDCVFIKSHGLVR 1173
                         *:                                                          
    SP|P21675|TAF1_HUMAN ------------------------------------------------------------
    SP|Q86U86|PB1_HUMAN  PRVGRIEKVWVRDGAAYFYGPIFIHPEETEHEPTKMFYKKEVFLSNLEETCPMTCILGKC 1233

    SP|P21675|TAF1_HUMAN ------------------------------------------------------------
    SP|Q86U86|PB1_HUMAN  AVLSFKDFLSCRPTEIPENDILLCESRYNESDKQMKKFKGLKRFSLSAKVVDDEIYYFRK 1293

    SP|P21675|TAF1_HUMAN ------------------------------------------------------------
    SP|Q86U86|PB1_HUMAN  PIVPQKEPSPLLEKKIQLLEAKFAELEGGDDDIEEMGEEDSEVIEPPSLPQLQTPLASEL 1353

    SP|P21675|TAF1_HUMAN ------------------------------------------------------------
    SP|Q86U86|PB1_HUMAN  DLMPYTPPQSTPKSAKGSAKKEGSKRKINMSGYILFSSEMRAVIKAQHPDYSFGELSRLV 1413

    SP|P21675|TAF1_HUMAN ------------------------------------------------------------
    SP|Q86U86|PB1_HUMAN  GTEWRNLETAKKAEYEERAAKVAEQQERERAAQQQQPSASPRAGTPVGALMGVVPPPTPM 1473

    SP|P21675|TAF1_HUMAN ------------------------------------------------------------
    SP|Q86U86|PB1_HUMAN  GMLNQQLTPVAGMMGGYPPGLPPLQGPVDGLVSMGSMQPLHPGGPPPHHLPPGVPGLPGI 1533

    SP|P21675|TAF1_HUMAN ------------------------------------------------------------
    SP|Q86U86|PB1_HUMAN  PPPGVMNQGVAPMVGTPAPGGSPYGQQVGVLGPPGQQAPPPYPGPHPAGPPVIQQPTTPM 1593

    SP|P21675|TAF1_HUMAN ------------------------------------------------------------
    SP|Q86U86|PB1_HUMAN  FVAPPPKTQRLLHSEAYLKYIEGLSAESNSISKWDQTLAARRRDVHLSKEQESRLPSHWL 1653

    SP|P21675|TAF1_HUMAN ------------------------------------
    SP|Q86U86|PB1_HUMAN  KSKGAHTTMADALWRLRDLMLRDTLNIRQAYNLENV 1689
    
Results from **UNIPROT** show the hidden Bromo motives in the region:

    SP|P21675|TAF1_HUMAN KIVLGKQLIESADEVRRKSLVLKFPKQQLPPKKKRRVGTTVHC----D---YLNRPHKSI 1374
    SP|Q86U86|PB1_HUMAN  AIY--KRVLK-------LQQVMQAKKKELARRDDIEDGDSMISSATSDTGSAKRKSKKNI 518
                          *   *::::        . *::  *::*  :.. . * :: .    *     .: :*.*
                          
More specifically:

    KIVLGKQLIESADEVRRKSLVLKFPKQQLPPKKKRRVGTTVHC----D---YLNRPHKSI 1374
                                 ^^^^^^^^

This result shows that there is a large offset from our reduced PSSM and yet a stable similarity to our un-reduced PSSM.

A point to make is that the secondary domain found by our PSSM is not found by **UNIPROT**.

Given the state of the art technology and resources implemented on their side, we shall assume this is due to an optimisation in their algorithms and search methods that we were unable to accomplish.